In [ ]:
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject
import numpy as np
import matplotlib.pyplot as plt

# File paths
files = {
    "DEM": "Filled_DEM.tif",
    "Flow Accumulation": "Flow_Acu_1.tif",
    "Slope": "Slope.tif",
    "TWI": "TWI.tif",
    "LULC": "LULC.tif",
    "Lineament Density": "Lineament_Density.tif"
}

# Load and print metadata
meta_info = {}

for name, path in files.items():
    with rasterio.open(path) as src:
        print(f"📌 {name}")
        print(f"  CRS: {src.crs}")
        print(f"  Resolution: {src.res}")
        print(f"  Bounds: {src.bounds}")
        print(f"  Shape: {src.shape}")
        print()
        meta_info[name] = {
            "crs": src.crs,
            "transform": src.transform,
            "shape": src.shape
        }


In [ ]:
import rasterio
from rasterio.warp import reproject, Resampling

with rasterio.open("Filled_DEM.tif") as dem_src:
    dem_crs = dem_src.crs
    dem_transform = dem_src.transform
    dem_shape = dem_src.shape
    dem_profile = dem_src.profile


In [ ]:
def reproject_to_dem(src_path, dst_path, dem_crs, dem_transform, dem_shape):
    with rasterio.open(src_path) as src:
        src_data = src.read(1)
        src_nodata = src.nodata
        dst_data = np.full(dem_shape, src_nodata if src_nodata is not None else -9999, dtype=src_data.dtype)
        
        reproject(
            source=src_data,
            destination=dst_data,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=dem_transform,
            dst_crs=dem_crs,
            resampling=Resampling.nearest
        )
        
        profile = dem_profile.copy()
        profile.update({
            "dtype": dst_data.dtype,
            "count": 1,
            "nodata": src_nodata if src_nodata is not None else -9999
        })
        
        with rasterio.open(dst_path, 'w', **profile) as dst:
            dst.write(dst_data, 1)

# Reproject files
reproject_to_dem("LULC.tif", "LULC_aligned.tif", dem_crs, dem_transform, dem_shape)
reproject_to_dem("Lineament_Density.tif", "Lineament_Density_aligned.tif", dem_crs, dem_transform, dem_shape)


In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

# ✅ Step 1: Extend the raster list
rasters = {
    "DEM": "Filled_DEM.tif",
    "LULC": "LULC_aligned.tif",
    "Lineament Density": "Lineament_Density_aligned.tif",
    "Flow Accumulation": "Flow_Acu_1.tif",
    "Slope": "Slope.tif",
    "TWI": "TWI.tif"
}

# ✅ Step 2: Set custom color maps and units
cmap_dict = {
    "DEM": "terrain",
    "LULC": "tab20",             # categorical
    "Lineament Density": "YlGnBu",
    "Flow Accumulation": "cubehelix",
    "Slope": "inferno",
    "TWI": "viridis"
}

units = {
    "DEM": "meters",
    "LULC": "category ID",
    "Lineament Density": "km/km²",
    "Flow Accumulation": "cells",
    "Slope": "degrees",
    "TWI": "index"
}

# ✅ Step 3: Define function to clean and display raster info
def display_and_clean_raster(name, path):
    with rasterio.open(path) as src:
        data = src.read(1).astype('float32')
        crs = src.crs
        shape = src.shape
        bounds = src.bounds
        transform = src.transform
        nodata = src.nodata

        # Step 4: Apply cleaning
        if nodata is not None:
            data[data == nodata] = np.nan
        data[data < -1e6] = np.nan
        data[data > 1e8] = np.nan  # extreme outlier threshold

        # Compute stats
        min_val = np.nanmin(data)
        max_val = np.nanmax(data)

        # Print summary
        print(f"📌 {name}")
        print(f"  Path: {path}")
        print(f"  CRS: {crs}")
        print(f"  Dimensions: {shape}")
        print(f"  Bounds: {bounds}")
        print(f"  Transform: {transform}")
        print(f"  Nodata value: {nodata}")
        print(f"  Value range: min = {min_val:.2f}, max = {max_val:.2f}")
        print("-" * 70)

        # Step 5: Visualize
        plt.figure(figsize=(8, 6))
        plt.imshow(data, cmap=cmap_dict[name], interpolation='none')
        plt.title(f"{name} ({units[name]})")
        plt.colorbar(label=units[name])
        plt.axis("off")
        plt.show()

        return data

# ✅ Step 6: Process all rasters
cleaned_data = {}
for name, path in rasters.items():
    cleaned_data[name] = display_and_clean_raster(name, path)


In [ ]:
print("\n✅ VERIFICATION OF CLEANED RASTERS\n")

# Define a function to check NaNs and valid range
def verify_cleaned_raster(name, data, expected_shape=None):
    print(f"🔎 {name}")
    # Check shape
    if expected_shape is not None:
        if data.shape != expected_shape:
            print(f"  ⚠️ Shape mismatch: {data.shape} (expected {expected_shape})")
        else:
            print(f"  ✅ Shape OK: {data.shape}")
    else:
        print(f"  Shape: {data.shape}")
    
    # Check NoData
    nan_count = np.isnan(data).sum()
    total_count = data.size
    nan_percent = (nan_count / total_count) * 100
    print(f"  🕳️ NaN count: {nan_count} ({nan_percent:.2f}%)")

    # Check range of remaining values
    valid_data = data[~np.isnan(data)]
    print(f"  🔢 Value range: min = {np.min(valid_data):.2f}, max = {np.max(valid_data):.2f}")

    # Check extreme residuals
    if (valid_data < -1e6).any() or (valid_data > 1e8).any():
        print("  ⚠️ Detected extreme/unexpected values. Further cleaning may be needed.")
    else:
        print("  ✅ No extreme values detected.")
    
    print("-" * 60)

# Use DEM as shape reference
ref_shape = cleaned_data["DEM"].shape

# Verify each cleaned layer
for name, data in cleaned_data.items():
    verify_cleaned_raster(name, data, expected_shape=ref_shape)


In [ ]:
import numpy as np
import rasterio
import zarr
from matplotlib import pyplot as plt

# Raster file paths
raster_paths = {
    "DEM": "Filled_DEM.tif",
    "LULC": "LULC_aligned.tif",
    "Lineament Density": "Lineament_Density_aligned.tif",
    "Flow Accumulation": "Flow_Acu_1.tif",
    "Slope": "Slope.tif",
    "TWI": "TWI.tif"
}

# Load and clean rasters
cleaned_data = {}
metadata = {}

for name, path in raster_paths.items():
    with rasterio.open(path) as src:
        data = src.read(1).astype("float32")
        if src.nodata is not None:
            data[data == src.nodata] = np.nan
        data[data < -1e6] = np.nan
        data[data > 1e8] = np.nan

        cleaned_data[name] = data
        metadata[name] = {
            "crs": src.crs.to_string(),
            "transform": src.transform,
            "shape": src.shape,
            "bounds": src.bounds,
            "nodata": src.nodata
        }


# Stack layers in order
layers_to_stack = [
    cleaned_data["DEM"],
    cleaned_data["Slope"],
    cleaned_data["Flow Accumulation"],
    cleaned_data["TWI"],
    cleaned_data["Lineament Density"],
   
]

static_stack = np.stack(layers_to_stack, axis=0)  # shape: (bands, H, W)

# Save to Zarr with full metadata
zarr_store = zarr.open("1_Static_data_alligned.zarr", mode='w',
                       shape=static_stack.shape,
                       chunks=(1, *static_stack.shape[1:]),
                       dtype='float32')

zarr_store[:] = static_stack
zarr_store.attrs["band_names"] = [
    "DEM", "Slope", "Flow Accumulation", "TWI", "Lineament Density"
]
zarr_store.attrs["crs"] = metadata["DEM"]["crs"]
zarr_store.attrs["transform"] = tuple(metadata["DEM"]["transform"])
zarr_store.attrs["bounds"] = tuple(metadata["DEM"]["bounds"])
zarr_store.attrs["nodata"] = metadata["DEM"]["nodata"]

print("✅ Saved to Zarr with full metadata")
print(f"📐 Shape: {static_stack.shape}")


In [ ]:
import numpy as np
import xarray as xr
import rioxarray
import matplotlib.pyplot as plt
import zarr
from affine import Affine

# ------------------------------------------------------------------------
# 1. Load the stacked Zarr array
zarr_path = "1_Static_data_alligned.zarr"
stack = zarr.load(zarr_path)  # shape: (7, rows, cols)
metadata = zarr.open(zarr_path).attrs

# Wrap it into an xarray DataArray
da = xr.DataArray(
    stack,
    dims=["band", "y", "x"],
    name="static_data"
)

# ------------------------------------------------------------------------
# 2. Extract the DEM band (band 0)
dem = da.isel(band=0)

# ------------------------------------------------------------------------
# 3. Read transform and bounds from Zarr metadata
transform_tuple = metadata["transform"]
transform = Affine(*transform_tuple)

bounds = metadata["bounds"]
crs = metadata["crs"]
nodata_val = metadata["nodata"]

# Get shape
height, width = dem.shape

# Generate coordinates using the affine transform
x_coords = (np.arange(width) + 0.5) * transform.a + transform.c
y_coords = (np.arange(height) + 0.5) * transform.e + transform.f

# Assign coordinates to the DataArray
dem = dem.assign_coords(x=x_coords, y=y_coords)
dem = dem.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=False)
dem = dem.rio.write_crs(crs, inplace=False)

# ------------------------------------------------------------------------
# 4. Optional: Set nodata if needed
if nodata_val is not None:
    dem = dem.where(dem != nodata_val)

# ------------------------------------------------------------------------
# 5. Print Metadata Summary
print("📌 DEM Metadata Info")
print(f"CRS            : {dem.rio.crs}")
print(f"Resolution     : {dem.rio.resolution()}")
print(f"Bounds         : {dem.rio.bounds()}")
print(f"Shape          : {dem.shape}")
print(f"NoData value   : {nodata_val}")
print(f"Min Elevation  : {float(dem.min()):.2f}")
print(f"Max Elevation  : {float(dem.max()):.2f}")

# ------------------------------------------------------------------------
# 6. Visualize DEM
plt.figure(figsize=(8, 6))
dem.plot(cmap='terrain')
plt.title("DEM from Zarr (with Embedded Metadata)")
plt.xlabel("Longitude or X (map units)")
plt.ylabel("Latitude or Y (map units)")
plt.grid(False)
plt.show()


In [ ]:
from pyproj import Transformer

# Get the shape
height, width = dem.shape

# Compute the centroid pixel index
centroid_row = height // 2
centroid_col = width // 2

# Extract x/y coordinate values using DataArray's coords
centroid_x = float(dem.x[centroid_col].values)
centroid_y = float(dem.y[centroid_row].values)

print("📍 DEM Centroid in Projected CRS:")
print(f"X (Easting) : {centroid_x}")
print(f"Y (Northing): {centroid_y}")

# Get CRS from DataArray
projected_crs = dem.rio.crs

# Set up transformer to WGS84 (EPSG:4326)
transformer = Transformer.from_crs(projected_crs, "EPSG:4326", always_xy=True)

# Convert to lat/lon
lon, lat = transformer.transform(centroid_x, centroid_y)

print("🌍 Converted Centroid Coordinates (WGS84):")
print(f"Longitude: {lon}")
print(f"Latitude : {lat}")


In [ ]:
import xarray as xr
import rioxarray
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from tqdm import tqdm
from affine import Affine

In [ ]:
# -----------------------
# Prepare rainfall files (2014–2024 only)
# -----------------------
rainfall_dir = r"F:\browse folder directory where data is present"
rainfall_files = sorted(glob.glob(os.path.join(rainfall_dir, "RF25_ind*_rfp25.nc")))

# Filter only 1994–2023 files
selected_files = [f for f in rainfall_files if "1994" <= os.path.basename(f)[8:12] <= "2023"]
print(f"🗂 Total files selected: {len(selected_files)}")

print(f"🗂 Selected files: {[os.path.basename(f) for f in selected_files]}\n")


In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import os
from pyproj import Transformer

# === Step 1: Extract DEM centroid (WGS84) ===
height, width = dem.shape
centroid_row = height // 2
centroid_col = width // 2
centroid_x = float(dem.x[centroid_col].values)
centroid_y = float(dem.y[centroid_row].values)
projected_crs = dem.rio.crs

transformer = Transformer.from_crs(projected_crs, "EPSG:4326", always_xy=True)
lon, lat = transformer.transform(centroid_x, centroid_y)

print("🌍 DEM Centroid in WGS84:")
print(f"Longitude: {lon}")
print(f"Latitude : {lat}")

# === Step 2: Extract rainfall at centroid across all years ===
rainfall_series = []
time_series = []

for file in selected_files:
    print(f"\n🔄 Processing: {os.path.basename(file)}")
    ds = xr.open_dataset(file)

    print("📦 Variables in dataset:", list(ds.data_vars.keys()))

    # Flexible keys
    lat_key = 'lat' if 'lat' in ds.coords else 'LATITUDE'
    lon_key = 'lon' if 'lon' in ds.coords else 'LONGITUDE'
    rain_key = 'rf' if 'rf' in ds.data_vars else 'RAINFALL'
    time_key = 'TIME' if 'TIME' in ds.coords else 'time'

    lat_vals = ds[lat_key].values
    lon_vals = ds[lon_key].values

    # Nearest grid index to centroid
    lat_idx = np.abs(lat_vals - lat).argmin()
    lon_idx = np.abs(lon_vals - lon).argmin()

    try:
        rf_daily = ds[rain_key].isel({lat_key: lat_idx, lon_key: lon_idx})
        rainfall_series.append(rf_daily.values)
        time_series.append(pd.to_datetime(ds[time_key].values))
        print("✅ Extracted daily rainfall (first 5):", rf_daily.values[:5])
    except Exception as e:
        print(f"❌ Failed to extract rainfall: {e}")

# === Step 3: Cross-Verification ===
print("\n🔎 Cross-Verification Summary:")
print(f"🟢 Years Processed     : {len(rainfall_series)}")
print(f"🟢 Total Days Extracted: {sum(len(r) for r in rainfall_series)}")
print(f"🟢 Sample Years with Non-Zero Rainfall:")

for i, rf in enumerate(rainfall_series):
    if np.any(rf > 0):
        print(f"  ✅ Year {i + 1994}: {rf[:5]} ...")

# === Step 4: Flatten + Stack for Zarr Output ===
rainfall_series_flat = np.concatenate(rainfall_series)
time_series_flat = np.concatenate(time_series)

# Create DataArray
rainfall_da = xr.DataArray(
    rainfall_series_flat,
    coords={"time": time_series_flat},
    dims=["time"],
    name="rainfall"
)

# Create Dataset and Save
rainfall_ds = xr.Dataset({"rainfall": rainfall_da})

output_zarr = "2_Dynamic_rainfall_centroid_1994_2023.zarr"
rainfall_ds.to_zarr(output_zarr, mode="w")

print(f"\n💾 Saved rainfall time series to: {output_zarr}")
